![Logo HS KL](https://upload.wikimedia.org/wikipedia/commons/thumb/5/5e/Logo_of_Hochschule_Kaiserslautern.png/320px-Logo_of_Hochschule_Kaiserslautern.png)
# Fuzz Testing
A Social Skills workshop by Jan

# Exercise: Advanced Fuzzing
Breaking things with not so random inputs.

## Task 1: Installation
Now that the basic concepts are clear, let's dive into a more advanced testing strategy!

In this exercise, you are going to test [libxml2](https://github.com/GNOME/libxml2), the XML parser library. It was originally developed for the GNOME project and is now available under almost all common operating systems.

Before the fun begins, you will have to install the required dependencies and build libxml2.

The *fuzzingbook* python module provides various Fuzzers and other useful tools for Fuzzing. It uses the *libgraphviz-dev* package and can be installed with the commands below:

In [ ]:
!apt install libgraphviz-dev
!pip install fuzzingbook

Download the archived version 2.9.4 of libxml2 and unpack it into directory */content/libxml2/libxml2-2.9.4*:

In [ ]:
!mkdir -p /content/libxml2 && cd /content/libxml2 && wget http://xmlsoft.org/download/libxml2-2.9.4.tar.gz -O - | tar -xzf - && echo -e "\n[Succesfully unpacked!]"

Configure the build of libxml2, build it with coverage information and AddressSanitizer enabled and install it into directory */content/libxml2/build*:

In [ ]:
!mkdir -p /content/libxml2/build && cd /content/libxml2/libxml2-2.9.4/ && CC=gcc CXX=g++ CFLAGS="--coverage -fsanitize=address" CXXFLAGS="--coverage -fsanitize=address" LDFLAGS="--coverage -fsanitize=address" ./configure --prefix="/content/libxml2/build" --disable-shared --without-debug --without-ftp --without-http --without-legacy --without-python LIBS='-ldl' && make -j4 && make install && echo -e "\n[Successfully installed!]"

You can test your installation by executing the *xmllint* program. It takes XML files, parses and prints them.

In [ ]:
!/content/libxml2/build/bin/xmllint --memory /content/libxml2/libxml2-2.9.4/test/valid/127772.xml

### 1.1 *gcov*
Since libxml2 was compiled with coverage information enabled, you can invoke the *gcov* program from the directory */content/libxml2/libxml2-2.9.4* to print this information for the specified source files. It will create coverage reports for these files, which show the percentage of lines executed.

In [ ]:
!cd /content/libxml2/libxml2-2.9.4 && gcov parser.c valid.c

If you are curious, you can also search for the files *parser.c.gcov* and *valid.c.gcov* in the directory */content/libxml2/libxml2-2.9.4*. They were also produced by *gcov* and include how many times each line was executed.

## Task 2: Fuzzing libxml2
You will now use the `GreyboxGrammarFuzzer` component of the *fuzzingbook* module to test libxml2. The goal is to execute as many lines in libxml2 as possible.

The code below defines a class, which executes external programs and measures their coverage with *gcov*:

In [6]:
import os, subprocess

from fuzzingbook.GreyboxGrammarFuzzer import *
from fuzzingbook.Fuzzer import ProgramRunner
from fuzzingbook.Coverage import read_gcov_coverage
from typing import Tuple, Union


class ProgramCoverageRunner(ProgramRunner):
  def __init__(self, program: Union[str, List[str]], project_root: str, source_files: List[str], temp_file: str) -> None:
    """Initialize.
    `program` is the program to test
    `project_root` is the root directory of the source files
    `source_files` are the source file names for which the coverage is to be measured
    `temp_file` is the file in which the generated input is written"""
    super().__init__(program)
    self.project_root = project_root
    self.source_files = source_files
    self.temp_file = temp_file
    self._coverage: Set[Location] = set()
    self._all_coverage: Set[Location] = set()
    self._cumulative_coverage: List[int] = []

  def run_process(self, inp: str) -> subprocess.CompletedProcess:
    """Run external program as a subprocess.
    `inp` is the input for the external program generated by the fuzzer"""
    try:
      with open(self.temp_file, 'w') as f:
        f.write(inp)
      result = subprocess.run(self.program.split() + [self.temp_file])
    except Exception as exc:
      self._coverage = self.compute_coverage()
      raise exc
    self._coverage = self.compute_coverage()
    self._all_coverage |= self._coverage
    self._cumulative_coverage.append(len(self._coverage))
    return result

  def compute_coverage(self) -> Set[Location]:
    """Compute line coverage with gcov."""
    coverage = set()
    cwd = os.getcwd()
    os.chdir(self.project_root)
    for source in self.source_files:
      if self.run_gcov(source):
        source_coverage = read_gcov_coverage(source)
        coverage.update(source_coverage)
    os.chdir(cwd)
    return coverage

  def run_gcov(self, source_file: str) -> bool:
    """Run gcov.
    `source_file` is the file with which gcov is executed"""
    result = subprocess.run(["gcov", source_file])
    if result.returncode != 0:
      return False
    return True

  def coverage(self) -> Set[Location]:
    """Return current coverage."""
    return self._coverage

  def cumulative_coverage(self) -> List[int]:
    """Return total coverage."""
    return self._cumulative_coverage

  def reset_coverage(self):
    """Reset all coverage information and delete gcov data files."""
    self._coverage.clear()
    self._all_coverage.clear()
    self._cumulative_coverage.clear()
    for source in self.source_files:
      gcov_file = os.path.join(self.project_root, source + ".gcov")
      gcda_file = os.path.join(self.project_root, source.split(".")[0] + ".gcda")
      try:
        print("Removing {}".format(gcov_file))
        os.remove(gcov_file)
      except FileNotFoundError:
        print("{} not found!".format(gcov_file))
        pass
      try:
        print("Removing {}".format(gcda_file))
        os.remove(gcda_file)
      except FileNotFoundError:
        print("{} not found!".format(gcda_file))
        pass



`GreyboxGrammarFuzzer` combines two methods to generate new inputs for the program under test:

1. Byte-based mutations
2. Region-based mutations

Both methods are examined in the following two sections.

### 2.1 Byte-based mutations
When performing byte-based mutations, a Fuzzer takes starting inputs - called seeds - and modifies their bytes to a certain degree to generate new inputs. Since you want to execute as many parts of the program under test as possible, the quality of the seeds is important. Random seeds like "!a7s\\)d" won't perform well when dealing with programs like XML parsers, which expect highly structural inputs.

In the code below, class `CustomMutator` derives three mutation functions from class `Mutator`, which modify an input in the following ways:
1. Insert a random character at a random position
2. Delete a random character at a random position, or do 1. if the input is empty
3. Flip a random bit at a random position

**Your job is to:**
* (optionally) Add custom mutation functions to class `CustomMutator`
* (optionally) Fill the `seeds` list with meaningful starting inputs

**Hint: Directory */content/libxml2/libxml2-2.9.4/test* may contain useful XML files**


In [7]:
import random


class CustomMutator(Mutator):
  def __init__(self):
    super().__init__()
    self.mutators += [

      # Your code goes here -->

      # Add the names of your custom mutators like:
      # self.custom_mutator

      # <--

    ]

  # Your code goes here -->

  # Define custom mutator functions, which take one string as their argument
  # and return a new, modified string, like:
  # def custom_mutator(self, s: str):
  #   new_s = do_crazy_stuff(s)
  #   return new_s

  # <--


seeds = [

# Your code goes here -->

"""
seed1
""",
"""
seed2
""",
"""
...
"""

# <--

]



### 2.2 Region-based Mutations
The idea of region-based mutation is to:
1. Parse an input into a tree based on a specified grammar
2. Identify regions in this tree associated with specific subtrees
3. Delete or swap these regions

**Your job is to:**
* (optionally) Expand the context-free grammar (A -> a | A = single nonterminal, a = string of terminals and/or nonterminals) for XML files, defined in variable `XML_GRAMMAR`

In [16]:
XML_TOKENS: Set[str] = {"<id>", "<text>"}


XML_GRAMMAR: Grammar = {
    "<start>":              ["<xml-tree>"],

    # Your code goes here -->

    "<xml-tree>":           ["<id>", "<text>"],
    "<id>":                 ["<letter>", "<id><letter>"],
    "<text>":               ["<text><letter_space>", "<letter_space>"],
    "<letter>":             srange(string.ascii_letters + string.digits +
                                   "\"" + "'" + "."),
    "<letter_space>":       srange(string.ascii_letters + string.digits +
                                   "\"" + "'" + " " + "\t"),

    # <--

}


# Check if XML_GRAMMAR is in a valid format
assert is_valid_grammar(XML_GRAMMAR)



### 2.3 Measuring coverage
Following code initializes all the components required for the test of libxml2. The external program which calls into libxml2 is *xmllint*. The call to *xmllint* with commandline arguments is defined in variable `program`.

In [17]:
import tempfile


# Earley Parser for your XML grammar
parser = EarleyParser(XML_GRAMMAR, tokens=XML_TOKENS)
# Region-based mutator
tree_mutator = RegionMutator(parser)
# Byte-based mutator
byte_mutator = CustomMutator()

# Scheduler to distribute fuzzing time among seeds in population
schedule = PowerSchedule()

# Fuzzer to perform mutations on seeds
fuzzer = GreyboxGrammarFuzzer(seeds, byte_mutator, tree_mutator, schedule)

# External program to use for calling into libxml2 including arguments
program = "/content/libxml2/build/bin/xmllint --memory --noenc --nocdata --dtdattr --loaddtd --valid --xinclude"
# Root directory of the source files
project_root = "/content/libxml2/libxml2-2.9.4"
# Source files for which the coverage is computed with gcov
sources = ["parser.c", "valid.c"]
# Temporary file to store inputs for external program
basename = "input.txt"
tempdir = tempfile.mkdtemp()
FILE = os.path.join(tempdir, basename)
# Runner to execute external program and compute coverage for each execution
runner = ProgramCoverageRunner(program, project_root, sources, FILE)



**Your job is to:**
* Execute the following code to start the test of libxml2 and plot the reached coverage.

In [ ]:
import matplotlib.pyplot as plt


# Reset all coverage information of previous runs
runner.reset_coverage()
# Start the test
outcome = fuzzer.runs(runner, trials=100)

# Get the reached coverage and plot it
coverage = runner.cumulative_coverage()
plt.plot(coverage, label="Maximum coverage: {}".format(max(coverage)))
plt.title('Coverage over time')
plt.xlabel('Number of inputs')
plt.ylabel('Lines covered')
plt.legend(loc="lower right")



## Additional Links
* [The Fuzzing Book](https://www.fuzzingbook.org/)
* [Smart Greybox Fuzzing](https://arxiv.org/abs/1811.09447)